### Predict
This example predicts the class for each sample given in the test dataset. We assume training is has already been completed using the training code, and the variables are located in the correct path.

If you get errors when running this code, restart the kernel and clear the outputs, then run all cells. You may also need to delete the traning data in /data/tensorflow_sessions and re-train.

Import framework and modules

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
from include.model import model
from include.data import get_data_set

Import (Create Model), Specify 6 classes to detect

In [2]:
x, y, output, global_step, y_pred_cls = model(6)

Instructions for updating:
Use the `axis` argument instead


Load dataset

In [3]:
test_x, test_y = get_data_set()
labels = ["Relax", "Ok", "Fist", "Like", "Rock", "Spock"]

Init Saver to save (checkpoint) training values (we're loading in this case to predict)

In [4]:
saver = tf.train.Saver()
_SAVE_PATH = "./data/tensorflow_sessions/myo_armband/"

sess = tf.Session()

try:
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=_SAVE_PATH)
    print("Trying to restore last checkpoint from: ", last_chk_path)
    saver.restore(sess, save_path=last_chk_path)
    print("Restored checkpoint from:", last_chk_path)
except Exception as e:
    print("Failed to restore checkpoint. Initializing variables instead.")
    print(e)
    sess.run(tf.global_variables_initializer())

Trying to restore last checkpoint from:  ./data/tensorflow_sessions/myo_armband/-20000
INFO:tensorflow:Restoring parameters from ./data/tensorflow_sessions/myo_armband/-20000
Restored checkpoint from: ./data/tensorflow_sessions/myo_armband/-20000


Predict labels for entire dataset

In [5]:
i = 0
predicted_class = np.zeros(shape=len(test_x), dtype=np.int)
while i < len(test_x):
    j = min(i + 300, len(test_x))
    batch_xs = test_x[i:j, :]
    batch_ys = test_y[i:j, :]
    predicted_class[i:j] = sess.run(y_pred_cls, feed_dict={x: batch_xs, y: batch_ys})
    i = j

Calculate Accuracy

In [6]:
correct = (np.argmax(test_y, axis=1) == predicted_class)
acc = correct.mean()*100
correct_numbers = correct.sum()
print("Accuracy on Test-Set: {0:.2f}% ({1} / {2})".format(acc, correct_numbers, len(test_x)))

Accuracy on Test-Set: 87.49% (17125 / 19573)


Create and Display Confusion Matrix

In [7]:
cm = confusion_matrix(y_true=np.argmax(test_y, axis=1), y_pred=predicted_class)
for i in range(6):
    class_name = "({}) {}".format(i, labels[i])
    print(cm[i, :], class_name)
class_numbers = [" ({0})".format(i) for i in range(6)]
print("".join(class_numbers))

[2039   76   64   72   31  200] (0) Relax
[  32 2230  266   22   20  144] (1) Ok
[  36  246 4611   46   27   86] (2) Fist
[  44   15   31 3802  170  143] (3) Like
[  18   12   28  185 1594   43] (4) Rock
[  75  103   76   91   46 2849] (5) Spock
 (0) (1) (2) (3) (4) (5)


Close tf session

In [8]:
sess.close()